In [5]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from numpy import genfromtxt
from metrics import Metrics
import csv, sys
import time

/home/sarcasm/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
train_dataset_small = genfromtxt('train_dataset_1400.csv', delimiter=',')
train_dataset_medium = genfromtxt('train_dataset_14000.csv', delimiter=',')
train_dataset_large = genfromtxt('train_dataset_49000.csv', delimiter=',')
cross_validation_dataset = genfromtxt('validation_dataset.csv', delimiter=',')
test_dataset = genfromtxt('test_dataset.csv', delimiter=',')

# Shuffle Train Data
np.random.shuffle(train_dataset_small)
np.random.shuffle(train_dataset_medium)
np.random.shuffle(train_dataset_large)

# Split into Inputs and Labels
train_inputs_small, train_labels_small = np.hsplit(train_dataset_small, 2)
train_inputs_small = np.array(train_inputs_small).reshape(len(train_dataset_small), 34)
train_labels_small = np.array(train_labels_small).reshape(len(train_dataset_small), 34)

train_inputs_medium , train_labels_medium  = np.hsplit(train_dataset_medium, 2)
train_inputs_medium = np.array(train_inputs_medium).reshape(len(train_dataset_medium), 34)
train_labels_medium = np.array(train_labels_medium).reshape(len(train_dataset_medium), 34)

train_inputs_large, train_labels_large = np.hsplit(train_dataset_large, 2)
train_inputs_large = np.array(train_inputs_large).reshape(len(train_dataset_large), 34)
train_labels_large = np.array(train_labels_large).reshape(len(train_dataset_large), 34)

cross_validation_inputs, cross_validation_labels = np.hsplit(cross_validation_dataset, 2)
cross_validation_inputs = np.array(cross_validation_inputs).reshape(len(cross_validation_dataset), 34)
cross_validation_labels = np.array(cross_validation_labels).reshape(len(cross_validation_dataset), 34)

test_inputs, test_labels = np.hsplit(test_dataset, 2)
test_inputs = np.array(test_inputs).reshape(len(test_dataset), 34)
test_labels = np.array(test_labels).reshape(len(test_dataset), 34)

# Normalize Data
gigabit_max_throughput = 125 * 1e6 # 1000 MB/s
gigabit_min_throughput = 1e2       # 100 bytes/s
norm = gigabit_max_throughput - gigabit_min_throughput

train_inputs_small = (train_inputs_small - gigabit_min_throughput) / norm
train_inputs_medium = (train_inputs_medium - gigabit_min_throughput) / norm
train_inputs_large = (train_inputs_large - gigabit_min_throughput) / norm
cross_validation_inputs = (cross_validation_inputs - gigabit_min_throughput) / norm
test_inputs = (test_inputs - gigabit_min_throughput) / norm

train_data_small = np.concatenate((train_inputs_small, train_labels_small), axis = 1)
train_data_medium = np.concatenate((train_inputs_medium, train_labels_medium), axis = 1)
train_data_large = np.concatenate((train_inputs_large, train_labels_large), axis = 1)
validation_data = np.concatenate((cross_validation_inputs, cross_validation_labels), axis = 1)
test_data = np.concatenate((test_inputs, test_labels), axis = 1)

In [17]:
print(test_data[0])

[0.29838456 0.21155664 0.43844472 0.94801352 0.52507151 0.28438229
 0.07966445 0.4462371  0.20474213 0.80705589 0.08195367 0.85242344
 0.15072627 0.89893003 0.85411628 0.32762067 0.74916412 0.09128691
 0.38495138 0.37555857 0.04604257 0.75324191 0.98375752 0.83147088
 0.99598586 0.74641631 0.81665583 0.46277895 0.13969151 0.54697761
 0.84922143 0.78215138 0.38294673 0.13838877 0.         0.
 0.         1.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         1.
 0.         1.         0.         0.         0.         1.
 0.         1.         0.         0.         0.         0.
 0.         0.        ]


In [18]:
number_of_hidden_units = 2

model = BernoulliRBM(n_components=number_of_hidden_units)
model.fit(train_data_small)

BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=2, n_iter=10,
       random_state=None, verbose=0)